In [15]:
# data base api

import sqlite3
con=sqlite3.connect(r"C:\projects\ScaleyStuffs\uni\DAT2\Week 7 Practical\lab9db.db")
# lab9.db didn't exist before we ran this cell

In [16]:
# remember to run this cell before trying to call the functions defined in it from elsewhere in the Jupyter notebook

def gettingHeaders(tup):
    returning = [tu[0] for tu in tup]
    return str(tuple(returning))

def gettingRecords(recs):
    returning = ""
    for record in recs:
        returning += str(record) + "\n"
    return returning

def formattingPrintings():
    print(gettingHeaders(cursor.description) + "\n\n" + gettingRecords(cursor.fetchall()))

In [17]:
cursor=con.cursor()
cursor.execute("""
CREATE TABLE Instructor(
id INTEGER PRIMARY KEY NOT NULL,
name TEXT,
address TEXT
)
""")
con.commit()
# table called Instructor created in database - empty rn only though

OperationalError: table Instructor already exists

In [18]:
# inserting values - remember can't use """ """ in """ """
cursor.execute("""
INSERT INTO Instructor (id, name, address)
VALUES (1001, "JS", "Sheffield");
""")
cursor.execute("""
INSERT INTO Instructor (id, name, address)
VALUES (1002, "MN", "Leeds");
""")
cursor.execute("""
INSERT INTO Instructor (id, name, address)
VALUES (1003, "SK", "Derby");
""")
con.commit()

IntegrityError: UNIQUE constraint failed: Instructor.id

In [19]:
# cursor.execute("""DROP TABLE Learner""")  # resetting shenanigans hummies
cursor.execute("""
CREATE TABLE Learner(
learnerNo INTEGER PRIMARY KEY NOT NULL,
name TEXT,
address TEXT,
age INTEGER,
id INTEGER,

FOREIGN KEY(id) REFERENCES Instructor(id)
ON DELETE CASCADE
ON UPDATE CASCADE
ON INSERT CASCADE
)
""")
con.commit()
cursor.execute("""
INSERT INTO Learner
VALUES(1, "TK", "York", 19, 1001)
""")
cursor.execute("""
INSERT INTO Learner
VALUES(2, "KS", "Newcastle", 20, 1002)
""")
cursor.execute("""
INSERT INTO Learner
VALUES(3, "FB", "Derby", 19, 1001)
""")
cursor.execute("""
INSERT INTO Learner
VALUES(4, "MD", "York", 18, 1003)
""")
con.commit()
# they're all kids! T.T I wanna have lessons and pass soon too, inshallah X'(

OperationalError: table Learner already exists

In [20]:
cursor.execute("""SELECT * FROM Learner WHERE learnerNo=1""")
formattingPrintings()

('learnerNo', 'name', 'address', 'age', 'id')

(1, 'TK', 'York', 19, 999)



In [21]:
# cursor.execute("DROP TABLE Lesson")
cursor.execute("""
CREATE TABLE Lesson(
lessonID INTEGER PRIMARY KEY NOT NULL,
date TEXT,
learnerNo INTEGER NOT NULL,

FOREIGN KEY(learnerNo) REFERENCES Learner(learnerNo)
ON DELETE CASCADE
ON UPDATE CASCADE
ON INSERT CASCADE
)
""")
con.commit()
cursor.execute("""
INSERT INTO Lesson
VALUES(111, "23-10-20", 1)
""")
cursor.execute("""
INSERT INTO Lesson
VALUES(222, "25-10-20", 2)
""")
cursor.execute("""
INSERT INTO Lesson
VALUES(333, "05-11-20", 1)
""")
cursor.execute("""
INSERT INTO Lesson
VALUES(444, "27-10-20", 3)
""")
cursor.execute("""
INSERT INTO Lesson
VALUES(554, "29-10-20", 4)
""")
con.commit()
cursor.execute("SELECT * FROM Lesson")
formattingPrintings()

OperationalError: table Lesson already exists

In [22]:
# before closing a connection, where applicable, make sure to commit, otherwise changes made won't be saved

# We are going to an insert a new instructor in the Instructor table, with ID=1004,name=SG, and address = Hull
cursor.execute("""
INSERT INTO Instructor
VALUES(?, ?, ?)""",
(1004, "SG", "Hull"))
# paramaterised insertions
con.commit()
cursor.execute("SELECT * FROM Instructor")
formattingPrintings()

('id', 'name', 'address')

(888, 'SG', 'Hull')
(999, 'JS', 'Sheffield')
(1001, 'JS', 'Sheffield')
(1002, 'MN', 'Leeds')
(1003, 'SK', 'Derby')
(1004, 'SG', 'Hull')



In [23]:
# Next, we are going to change the id of one of the instructors. Current id = 1001, and needs to be changed to 999.

# cursor.execute("""
# UPDATE Instructor
# SET id=1001
# WHERE id=999
# """)
con.execute("PRAGMA foreign_keys=ON")  # no idea what this does, works the same without in my limited understanding XOS
# wai without this statement, foreign key id in Learner table is not updated - ooo
# through this statement, foreign keys are enforced in this relational database
cursor.execute("""
UPDATE Instructor
SET id=999
WHERE id=1001
""")
con.commit()
cursor.execute("SELECT * FROM Instructor")
formattingPrintings()
cursor.execute("SELECT * FROM Learner")
formattingPrintings()
# checking to see if all corresponding tables have been updated

IntegrityError: UNIQUE constraint failed: Instructor.id

In [24]:
# Another way of using the UPDATE statement:
# Suppose we want to update the id of one of the instructors, with current id=1004 to 888, we can use the following commands:

VALUES = (888, 1004)
cursor.execute("""
UPDATE Instructor
SET id=?
WHERE id=?
""", VALUES)
con.commit()
cursor.execute("SELECT * FROM Instructor")
formattingPrintings()

IntegrityError: UNIQUE constraint failed: Instructor.id

In [25]:
# Now, you need to delete a learner, with learnerNo = 3. Write your commands to perform this
# change and check if your table has been updated in the DB browser.
cursor.execute("""
DELETE FROM Learner WHERE learnerNo=3
""")
con.commit()
cursor.execute("""
SELECT learnerNo
FROM Learner
WHERE (SELECT learnerNo FROM Learner WHERE learnerNo=3)
""")
cursor.fetchall()  # returns empty list
# formattingPrintings()  # returns nothing due to bool - only column header learnerNo

[]

In [26]:
# We are now going to process results from SELECT using the fetchmany(size), fetchall(), and
# fetchone() methods.
# Suppose we want to retrieve all attributes and records from Instructor table. To accomplish this,
# we simply need the following commands.
cursor.execute("SELECT * FROM Instructor")
for row in cursor.fetchall():
    print(row[0], row[1], row[2])
# Now, if you only want the following columns - ‘id’ and ‘name’ – in your result, how would you
# adjust the above commands?
for row in cursor.fetchall():
    print(row[0], row[1])
    
# nicely formatted, much better than my just tuple printing function lollies XP

888 SG Hull
999 JS Sheffield
1001 JS Sheffield
1002 MN Leeds
1003 SK Derby
1004 SG Hull


In [27]:
# How would you use fetchone() method to calculate and display the average age?
import math as m
cursor.execute("SELECT AVG(age) FROM Learner")
row = cursor.fetchone()
print("Average age of all learners at this driving school is: ", m.floor(row[0]))

Average age of all learners at this driving school is:  19


In [14]:
# How would you use the fetchmany() method to display the first two records from the Learner table?
cursor.execute("SELECT * FROM Learner")
tworows = cursor.fetchmany(2) # specific value of records to be returned passed through as parameter
for row in tworows:
    print(row[0], row[1], row[2], row[3], row[4])

1 TK York 19 999
2 KS Newcastle 20 1002
